In [1]:
%aiida

Loaded AiiDA DB environment - profile name: default.

In [2]:
import ipywidgets as ipw

In [5]:
from aiida.orm import WorkChainNode
from aiida import orm
from itertools import combinations
from aiida_sssp_workflow.calculations.calculate_bands_distance import get_bands_distance
from aiida_sssp_workflow.utils import NONMETAL_ELEMENTS

def bands_distance(element):
    qb = QueryBuilder()
    qb.append(
        WorkChainNode,
        filters={
            'and':[
                {'attributes.exit_status': 0},
                {'attributes.process_label': 'VerificationWorkChain'},
                {'extras.element': element},
            ]
        },
    )
    all_results = qb.all(flat=True)
    
    if element in NONMETAL_ELEMENTS:
        is_metal = False
    else:
        is_metal = True
    
    results = []
    for pairs in combinations(all_results, 2):
        extras0 = pairs[0].extras
        extras1 = pairs[1].extras
        
        pp_info0 = f"{extras0['pp_type']}-{extras0['pp_family']}-{extras0['pp_version']}"
        pp_info1 = f"{extras1['pp_type']}-{extras1['pp_family']}-{extras1['pp_version']}"
        
        bandsdata_a = pairs[0].outputs.band_structure__seekpath_band_structure__band_structure
        band_parameters_a = pairs[0].outputs.band_structure__seekpath_band_structure__band_parameters
        bandsdata_b = pairs[1].outputs.band_structure__seekpath_band_structure__band_structure
        band_parameters_b = pairs[1].outputs.band_structure__seekpath_band_structure__band_parameters
        
        res = get_bands_distance(bandsdata_a, bandsdata_b,
                               band_parameters_a, band_parameters_b,
                               0.1, # 0.1eV smearing F-D
                               is_metal)
        print(f'The bands different between {pp_info0} and {pp_info1} is:')
        
        print(f'eta_v: {res.get("eta_v")}')
        print(f'shift_v: {res.get("shift_v")}')
        print(f'max_diff_v: {res.get("max_diff_v")}')
        print(f'eta_10: {res.get("eta_10")}')
        print(f'shift_10: {res.get("shift_10")}')
        print(f'max_diff_10: {res.get("max_diff_10")}')
        
        result = {
            'Band_A': pp_info0,
            'Band_B': pp_info1,
            'eta_v': res.get("eta_v"),
            'eta_10': res.get("eta_10"),
            'max_diff_v': res.get("max_diff_v"),
            'max_diff_10': res.get("max_diff_10"),
        }
        results.append(result)
    return results

In [7]:
description = ipw.HTML("""<p>Give the element and click 'Go'</p>""")
element = ipw.Text(description='element:')
button = ipw.Button(description="Go")
output = ipw.Output()

choose_element = ipw.VBox(children=[description, element, button])

def on_button_clicked(_):
    output.clear_output()
    with output:
        results = bands_distance(element.value)
        print(results)
        
button.on_click(on_button_clicked)

display(choose_element, output)

Output()